In [1]:
from timeit import default_timer as timer
import itertools
import os
import sys
import uuid
from glob import glob
from pathlib import Path
import json
import tweepy
import numpy as np
import pandas as pd
import multiprocessing as mp
import psutil
import socket
from functools import partial
import pyarrow.parquet as pq

# Params

In [2]:
cutoff = 1000
print('Save Data After Downloading',cutoff,'Timelines')

Save Data After Downloading 1000 Timelines


In [3]:
def get_env_var(varname,default):
    
    if os.environ.get(varname) != None:
        var = int(os.environ.get(varname))
        print(varname,':', var)
    else:
        var = default
        print(varname,':', var,'(Default)')
    return var

# Choose Number of Nodes To Distribute Credentials: e.g. jobarray=0-4, cpu_per_task=20, credentials = 90 (<100)
SLURM_JOB_ID            = get_env_var('SLURM_JOB_ID',0)
SLURM_ARRAY_TASK_ID     = get_env_var('SLURM_ARRAY_TASK_ID',0)
SLURM_ARRAY_TASK_COUNT  = get_env_var('SLURM_ARRAY_TASK_COUNT',1)
SLURM_JOB_CPUS_PER_NODE = get_env_var('SLURM_JOB_CPUS_PER_NODE',mp.cpu_count())

SLURM_JOB_ID : 0 (Default)
SLURM_ARRAY_TASK_ID : 0 (Default)
SLURM_ARRAY_TASK_COUNT : 1 (Default)
SLURM_JOB_CPUS_PER_NODE : 16 (Default)


In [4]:
path_to_data='../data'
path_to_users_ids = os.path.join(path_to_data + '/users')
path_to_users_old = os.path.join(path_to_data,'timelines','API','all_users_ids')
path_to_users_recent = os.path.join(path_to_data,'timelines','API','most_recent_id')
path_to_keys = os.path.join('../keys')
path_to_timelines = os.path.join(path_to_data,'timelines','API','IDF-updates')
os.makedirs(path_to_timelines, exist_ok=True)
print(path_to_users_recent)
print(path_to_keys)
print(path_to_timelines)

../data/timelines/API/most_recent_id
../keys
../data/timelines/API/IDF-updates


# Credentials

In [5]:
def get_key_files(SLURM_ARRAY_TASK_ID,SLURM_ARRAY_TASK_COUNT,SLURM_JOB_CPUS_PER_NODE):

    # Randomize set of key files using constant seed
    np.random.seed(0)
    all_key_files = list(np.random.permutation(glob(os.path.join(path_to_keys,'key*'))))
   # all_key_files.reverse()
    auth_file = np.random.permutation(glob(os.path.join(path_to_keys,'auth*')))
    
    # Split file list by node
    key_files = np.array_split(all_key_files,SLURM_ARRAY_TASK_COUNT)[SLURM_ARRAY_TASK_ID]
    
    # Check that node has more CPU than key file 
    if len(key_files) <= SLURM_JOB_CPUS_PER_NODE:
        print('# Credentials Allocated To Node:', len(key_files)) 
    else:
        print('Check environment variables:')
        print('# Credentials (',len(key_files),') > # CPU (', SLURM_JOB_CPUS_PER_NODE,')')
        print('Only keeping', SLURM_JOB_CPUS_PER_NODE, 'credentials')
        key_files = key_files[:SLURM_JOB_CPUS_PER_NODE]
        
    return key_files, auth_file

key_files, auth_file = get_key_files(SLURM_ARRAY_TASK_ID,SLURM_ARRAY_TASK_COUNT,SLURM_JOB_CPUS_PER_NODE)
print('\n'.join(key_files))
print('\n'.join(auth_file))

# Credentials Allocated To Node: 15
../keys/key_cyril.json
../keys/key_noemie.json
../keys/key_othmane.json
../keys/key_sam.json
../keys/key_liubov.json
../keys/key_naila.json
../keys/key_jihanne.json
../keys/key_chakresh.json
../keys/key_charlotte.json
../keys/key_youssr.json
../keys/key_felix.json
../keys/key_marc.json
../keys/key_clemence.json
../keys/key_naila2.json
../keys/key_zoe.json
../keys/auth_naila.json


In [6]:
def get_auth(key_file):
    
    # Import Auth keys
    for auth_file in glob(os.path.join(path_to_keys,'auth*')) :
        with open (auth_file) as f:
            auth_key = json.load(f)
    
    # Import token pairs
    with open(key_file) as f:
        key = json.load(f)

    # OAuth process, using the keys and tokens
    auth = tweepy.OAuthHandler(auth_key['consumer_key'], auth_key['consumer_secret'])
    auth.set_access_token(key['access_token'], key['access_token_secret'])

    # Creation of the actual interface, using authentication
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    
    try:
        api.verify_credentials()
        print(key_file,": Authentication checked")
    except:
        print(key_file,": error during authentication")
        sys.exit('Exit')
    
    return auth, api

In [7]:
for file in key_files:
    get_auth(file)

../keys/key_cyril.json : Authentication checked
../keys/key_noemie.json : Authentication checked
../keys/key_othmane.json : Authentication checked
../keys/key_sam.json : Authentication checked
../keys/key_liubov.json : Authentication checked
../keys/key_naila.json : Authentication checked
../keys/key_jihanne.json : Authentication checked
../keys/key_chakresh.json : Authentication checked
../keys/key_charlotte.json : Authentication checked
../keys/key_youssr.json : Authentication checked
../keys/key_felix.json : Authentication checked
../keys/key_marc.json : Authentication checked
../keys/key_clemence.json : Authentication checked
../keys/key_naila2.json : Authentication checked
../keys/key_zoe.json : Authentication checked


# Users list

In [8]:
start = timer()
print('Select Users...')
users_old=pq.ParquetDataset([x for x in Path(path_to_users_old).glob("**/*.parquet")]).read().to_pandas()
users_recent=pq.ParquetDataset([x for x in Path(path_to_users_recent).glob("**/*.parquet")]).read().to_pandas()
#users=pq.ParquetDataset(glob(os.path.join(path_to_users,'*.parquet'))).read().to_pandas()

users=pd.concat([users_old, users_recent])
users=users.sort_values(by=['user_id','created_at'], ascending=True)
users=users.drop_duplicates(subset='user_id', keep='last')

# Randomize users
users=users.sample(frac=1,random_state=0)

print('# Users :', len(users)) 

end = timer()
print('Computing Time:', round(end - start), 'sec')

Select Users...
# Users : 30651
Computing Time: 1 sec


In [9]:
start = timer()
print('Remove users whose timeline were successfully downloaded...')

def get_success():
    
    if not os.path.exists(os.path.join(path_to_timelines, 'success')):
        return set()
    else:
        success = set()
        with open(os.path.join(path_to_timelines, 'success'), 'r', encoding='utf-8') as file:
            for line in file:
                success.add(line.strip('\n').split('\t')[0])
        return set(success)

success=get_success()
print('# downloaded timelines:', len(success))

users=users[-users.user_id.isin(success)].copy()

end = timer()
print('Computing Time:', round(end - start), 'sec')

Remove users whose timeline were successfully downloaded...
# downloaded timelines: 0
Computing Time: 0 sec


## Get timelines

In [10]:
def get_timeline(user_id,tweet_id,api):
    
    timeline = []
    error = None
    
    # Collect All Statuses in Timeline
    try:
        cursor = tweepy.Cursor(
        api.user_timeline, 
        user_id=user_id, 
        since_id=tweet_id,
        count=3200,
        tweet_mode="extended", 
        include_rts=True).items()
        
        for status in cursor:
            timeline.append(status._json)
     
    except tweepy.error.TweepError as e:
        error = str(e)
        
    timeline = pd.DataFrame(timeline, columns = ['id_str','user','full_text','created_at','lang','place'])
    timeline['user_id'] = timeline['user'].apply(lambda x: x.get('id_str'))
    timeline['user_name'] = timeline['user'].apply(lambda x: x.get('name'))
    timeline['country_code'] = timeline.loc[~timeline['place'].isna(), 'place'].apply(lambda x: x.get('country_code'))
    timeline['bounding_box'] = timeline.loc[~timeline['place'].isna(), 'place'].apply(lambda x: x.get('bounding_box'))
    #timeline['coordinates'] = timeline.loc[~timeline['bouding_box'].isna(), 'bouding_box'].apply(lambda x: x.get('coordinates'))
    timeline['city'] = timeline.loc[~timeline['place'].isna(), 'place'].apply(lambda x: x.get('full_name'))
    
    return timeline[['id_str','user_id','user_name','full_text','created_at','lang','country_code',
                     'city','bounding_box']], error


#timeline = get_timeline('12','1266367509055209473',get_auth(key_files[0])[1])

In [11]:
timeline = get_timeline('2606411695','1267575252898459653',get_auth(key_files[0])[1])

../keys/key_cyril.json : Authentication checked


In [12]:
timeline

(                 id_str     user_id                        user_name  \
 0   1288050186502242304  2606411695  Nicolas Marguerie Photographies   
 1   1283462230814711809  2606411695  Nicolas Marguerie Photographies   
 2   1283461388053209093  2606411695  Nicolas Marguerie Photographies   
 3   1280923643049119744  2606411695  Nicolas Marguerie Photographies   
 4   1280866923631304705  2606411695  Nicolas Marguerie Photographies   
 5   1280802757029625856  2606411695  Nicolas Marguerie Photographies   
 6   1280800442243563520  2606411695  Nicolas Marguerie Photographies   
 7   1280633383375290368  2606411695  Nicolas Marguerie Photographies   
 8   1278613949395668992  2606411695  Nicolas Marguerie Photographies   
 9   1278354085046075393  2606411695  Nicolas Marguerie Photographies   
 10  1278079343944765441  2606411695  Nicolas Marguerie Photographies   
 11  1278026876947255296  2606411695  Nicolas Marguerie Photographies   
 12  1277649623386140673  2606411695  Nicolas Margu

In [13]:
users

,id_str,created_at,full_text,lang,user_id,user_name,country_code,city,coordinates,bounding_box_type
43305,1271069157346459649,2020-06-11 13:17:57,RT @liamgallagher: Pints of lager c’mon Boris ...,en,283204345,Torabisu Sukotto,None,None,None,None
44557,1267184323251273728,2020-05-31 20:01:00,@LaurRin_ @CAgiusILD @DenoyelleW @WrestleMania...,und,563365714,dick riviere,FR,"Paris, France","[[[2.2241006, 48.8155214], [2.4699053, 48.8155...",Polygon
8409,1273016822665818112,2020-06-16 22:17:16,Mercredi de bonheur \n#BLACK_LIVES_MATTER http...,fr,883826000261459968,mack ten,FR,"Dammarie-les-Lys, France","[[[2.5848515, 48.4946347], [2.6551137, 48.4946...",Polygon
44893,1270604929992925184,2020-06-10 06:33:16,RT @spopow9: ALORS COMME ÇA QUAND C’EST UN REB...,fr,736811823429746688,shikamaru,None,None,None,None
33494,1266145402530693121,2020-05-28 23:12:42,"RT @Harvard: Today, Harvard awarded a total of...",en,202501528,Harvard Club France,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
6725,1285870131030167552,2020-07-22 09:31:44,RT @berarddemalavas: Mise à jour du FAQ de @Tr...,fr,240194563,Cgt Interforum Editis Informer-Alerter-Mobiliser,None,None,None,None
7740,1278791674362200066,2020-07-02 20:44:28,🤍 \n\n(🔜 #Calogero @calogerofficiel),und,401703279,cyrille sauzéat,None,None,None,None
6203,1285188824084160514,2020-07-20 12:24:27,RT @05_Arslanbekov: Hahahha ca m’tue à la fin ...,fr,1606669932,Aldina,None,None,None,None
6352,1285518247484366849,2020-07-21 10:13:28,RT @FloLecointre: Accueil #presse @VisitCoteda...,fr,1872787320,Nathalie Dalmasso,None,None,None,None


In [14]:
def download_timelines(index_key):

    # Create Access For Block of Users
    api = get_auth(key_files[index_key])[1]    
    
    # Select Block of Users
    users_block = np.array_split(users,len(key_files))[index_key][['user_id','id_str']].values.tolist()
    
    # Initialize Output File ID
    output_id = str(uuid.uuid4())
    
    # Initialize DataFrame
    timelines = pd.DataFrame()
    
    # Initialize Downloaded User List
    downloaded_ids = []
    
    for (user_id,tweet_id) in users_block:
        
        # Try Downloading Timeline
        timeline, error = get_timeline(user_id,tweet_id,api)
        
        if error!=None:
            print(user_id,index_key,error)
            continue
            
        # Append
        else:
            #print('Success : ' + str(user_id))
            timelines = pd.concat([timelines, timeline],sort=False)
            downloaded_ids.append(user_id)
            
        # Save after <cutoff> timelines or when reaching last user
        if len(downloaded_ids) == cutoff or user_id == users_block[-1][0]:
            
            filename = 'timelines-'+ str(SLURM_JOB_ID)+'-'+ str(SLURM_ARRAY_TASK_ID)+'-'+ str(index_key)+'-'+ \
            str(len(downloaded_ids))+'-'+ output_id+'.json.bz2'
            
            print('Process', index_key, 'saving', len(downloaded_ids), 'timelines with output file:', 
            os.path.join(path_to_timelines,filename))
            
            # Save as list of dict discarding index
            timelines.to_json(
            os.path.join(path_to_timelines,filename),
            orient='records',
            force_ascii=False,
            date_format=None,
            double_precision=15,
            compression='bz2')
            #print(timelines)
            
            # Save User Id and File In Which Its Timeline Was Saved
            with open(os.path.join(path_to_timelines,'success'), 'a', encoding='utf-8') as file:
                for downloaded_id in downloaded_ids:
                    file.write(downloaded_id+'\t'+filename+'\n')
            
            # Reset Output File ID, Data, and Downloaded Users
            del timelines, downloaded_ids
            output_id = str(uuid.uuid4())
            timelines = pd.DataFrame()
            downloaded_ids = []
            
    return 0

In [15]:
print('Extract Timelines...\n')
with mp.Pool() as pool:
    pool.map(download_timelines, range(len(key_files)))

Extract Timelines...

../keys/key_cyril.json../keys/key_jihanne.json ../keys/key_naila.json../keys/key_felix.json ../keys/key_naila2.json ../keys/key_charlotte.json../keys/key_clemence.json ../keys/key_noemie.json ../keys/key_chakresh.json../keys/key_sam.json../keys/key_marc.json../keys/key_liubov.json../keys/key_youssr.json: Authentication checked  : Authentication checked : Authentication checked   : Authentication checked
: Authentication checked
 : Authentication checked: Authentication checked: Authentication checked: Authentication checked
: Authentication checked
../keys/key_othmane.json
: Authentication checked


../keys/key_zoe.json  
 : Authentication checked

: Authentication checked

: Authentication checked: Authentication checked

1655477510 13 283204345 Twitter error response: status code = 4010 
Twitter error response: status code = 401
41245718 14 Twitter error response: status code = 401
936211104442675200 11 Twitter error response: status code = 401
1160526756 10 Twi

Rate limit reached. Sleeping for: 595


Twitter error response: status code = 401


Rate limit reached. Sleeping for: 595


2901606771 9 Twitter error response: status code = 401
1057765927557779456 10 Twitter error response: status code = 401


...

Rate limit reached. Sleeping for: 623
Rate limit reached. Sleeping for: 621
Rate limit reached. Sleeping for: 617
Rate limit reached. Sleeping for: 617
Rate limit reached. Sleeping for: 616
Rate limit reached. Sleeping for: 617
Rate limit reached. Sleeping for: 607
Rate limit reached. Sleeping for: 603


464763642 10 Twitter error response: status code = 401
960095245278109696 6 Twitter error response: status code = 401
1272136182999252992 6 Twitter error response: status code = 404
1249482629851648001 1 Twitter error response: status code = 401
1245457024093167617 6 Twitter error response: status code = 404
1242039574697836544 12 Twitter error response: status code = 401
1269887786607403009 12 Twitter error response: status code = 401
1416148094 11 Twitter error response: status code = 401
3579555262 7 Twitter error response: status code = 401
1197596642788216833 12 Twitter error response: status code = 401
4243263257 12 Twitter error response: status code = 401
776167533674172416 13 Twitter error response: status code = 401
134962843 2 Twitter error response: status code = 401
121114618 9 Twitter error response: status code = 401
1220121876468510720 4 Twitter error response: status code = 401
1247220976821075974 8 Twitter error response: status code = 401
1223415484315553793 2 Twitte

Process 13 saving 990 timelines with output file: ../data/timelines/API/IDF-updates/timelines-0-0-13-990-921a0650-fff5-4375-b633-246c879fcd6c.json.bz2
Process 1 saving 1000 timelines with output file: ../data/timelines/API/IDF-updates/timelines-0-0-1-1000-cb33a656-7e7b-4aca-ba39-8b61db05486b.json.bz2
Process 7 saving 991 timelines with output file: ../data/timelines/API/IDF-updates/timelines-0-0-7-991-332092ad-b8a1-4af1-9abe-c7a26cef0aef.json.bz2
3749396362 4 Twitter error response: status code = 401
Process 1 saving 3 timelines with output file: ../data/timelines/API/IDF-updates/timelines-0-0-1-3-b5004ddc-0b3d-45a8-91b7-08a39eacf1d4.json.bz2
Process 4 saving 1000 timelines with output file: ../data/timelines/API/IDF-updates/timelines-0-0-4-1000-bb5d3bc4-756b-4c87-9693-95fe8aa48651.json.bz2
1076624514031800321 5 Twitter error response: status code = 401
1219400642160209927 5 Twitter error response: status code = 401
Process 4 saving 4 timelines with output file: ../data/timelines/API/I

Rate limit reached. Sleeping for: 563


1247544995860615169 5 Twitter error response: status code = 401
1273939445956444163 5 Twitter error response: status code = 401
14640579 5 Twitter error response: status code = 404
